**Processing Data for White-Box Attack**


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# reinstall updated versions 
!pip install tensorflow
!pip install keras
!pip install cleverhans

In [ ]:
# create folder called ECGadv-master
# store the downloaded entire repository from https://github.com/codespace123/ECGadv in ECG_advmaster
# indices for all CORRECT classifications done by ECG classifying model

# download the training data (training2017) from https://archive.physionet.org/challenge/2017/training2017.zip.
# store in folder called ECG_v1
# create a folder in ECG_v1 called Processed Data

import csv
import numpy as np
import scipy.io

csvfile_a = np.array(list(csv.reader(open("/content/drive/My Drive/ECGadv-master/data_select_A.csv"))))[:,3]
csvfile_a = [int(i) for i in csvfile_a]
csvfile_n = np.array(list(csv.reader(open("/content/drive/My Drive/ECGadv-master/data_select_N.csv"))))[:,3]
csvfile_n = [int(i) for i in csvfile_n]
csvfile_o = np.array(list(csv.reader(open("/content/drive/My Drive/ECGadv-master/data_select_O.csv"))))[:,3]
csvfile_o = [int(i) for i in csvfile_o]
csvfile_i = np.array(list(csv.reader(open("/content/drive/My Drive/ECGadv-master/data_select_i.csv"))))[:,3]
csvfile_i = [int(i) for i in csvfile_i]
print(csvfile_a)

In [ ]:
# PROCESSING ALL THE TRAINING DATA TO BE ATTACKED

import scipy.io
from glob import glob
import numpy as np

from natsort import natsorted
files = natsorted(glob("/content/drive/My Drive/ECG_v1/training2017"+ "/*.mat"))
print(len(files))
data = np.zeros((8529, 9000,1))
num = 1
for path in files:
  if path.find("(") == -1:
    temp = scipy.io.loadmat(path)
    temp = np.array(temp["val"])
    temp = np.nan_to_num(temp)
    temp = temp[:,:9000]
    temp = temp - np.mean(temp)
    temp = temp / np.std(temp)
    if temp.shape[1] < 9000:
      temp = np.pad(temp,((0,0),(0,9000 - temp.shape[1])), 'constant')
    temp = np.expand_dims(temp, axis = 2)
    data[num] = temp
    num += 1
  if path.find("(") != -1:
    print("Duplicate")
    print(path)
print(num-1)
np.save("/content/drive/My Drive/ECGadv-master/ProcessedData/agg_data.npy", data)

In [ ]:
# Reload the combined dataset

data = np.load("/content/drive/My Drive/ECGadv-master/ProcessedData/agg_data.npy")

In [ ]:
# Separating out the individual ECG data from the aggregate combined dataset into 4 datasets

import numpy as np
from tensorflow.keras.models import load_model
model = load_model("/content/drive/My Drive/ECG_v1/ecg_mit.hdf5")
for i in csvfile_a:
  a = np.expand_dims(data[i,:,:],axis=0)
  a = np.append(a, [[[0]]], axis = 1)
 
  %cd
  %cd /content/drive/My Drive/ECGadv-master/ProcessedData/A
  np.save("A_{}".format(i),a)

for i in csvfile_n:
  n = np.expand_dims(data[i,:,:],axis=0)
  n = np.append(n, [[[1]]], axis = 1)

  %cd
  %cd /content/drive/My Drive/ECGadv-master/ProcessedData/N
  np.save("N_{}".format(i),n)

for i in csvfile_o:
  o = np.expand_dims(data[i,:,:],axis=0)
  o = np.append(o, [[[2]]], axis = 1)
 
  %cd
  %cd /content/drive/My Drive/ECGadv-master/ProcessedData/O
  np.save("O_{}".format(i),o)

for i in csvfile_i:
  t = np.expand_dims(data[i,:,:],axis=0)
  t = np.append(t, [[[3]]], axis = 1)
 
  %cd
  %cd /content/drive/My Drive/ECGadv-master/ProcessedData/Tilde
  
  np.save("Tilde_{}".format(i),t)



In [ ]:
dicti = {
     0 : "/content/drive/My Drive/ECGadv-master/ProcessedData/A",
     1 : "/content/drive/My Drive/ECGadv-master/ProcessedData/N",
     2 : "/content/drive/My Drive/ECGadv-master/ProcessedData/O",
     3 : "/content/drive/My Drive/ECGadv-master/ProcessedData/Tilde",
}

In [ ]:
# MAKING RANDOM PAIRS OF ORIGINAL AND TARGET DATA
# create folder in ECG_v1 called PairedDataPhysio
# download Classifying Model from https://github.com/fernandoandreotti/cinc-challenge2017/blob/master/deeplearn-approach/ResNet_30s_34lay_16conv.hdf5

import numpy as np
from glob import glob

from tensorflow.keras.models import load_model
model = load_model("/content/drive/My Drive/ECG_v1/ResNet_30s_34lay_16conv.hdf5")

x = 0
y = 0

for j in range(4):
  for k in range(4):
    x = j
    y = k
    if(x == y):
      continue
    %cd
    %cd /content/drive/My Drive/ECG_v1/PairedDataPhysio
    p = "ECG_MIT_orig" + "{}".format(x) +"_target"+ "{}".format(y)
    !mkdir $p
    %cd $p  

    for z in range(10):
      base_o = dicti[x]
      base_t = dicti[y]
      paths_o = sorted(glob(base_o + "/*.npy"))
      paths_t = sorted(glob(base_t + "/*.npy")) 
      num_o = np.random.randint(len(paths_o))
      num_t = np.random.randint(len(paths_t))
      o = np.load(paths_o[num_o])
      t = np.load(paths_t[num_t])
      if model.predict(o[:,:-1,:]).argmax() != o[:,-1,:]:
        print("You messed up.")
      if model.predict(t[:,:-1,:]).argmax() != t[:,-1,:]:
        print("You messed up v2")
      %cd
      %cd /content/drive/My Drive/ECG_v1/PairedDataPhysio
      %cd $p  
      d = "ECG_Pair{}".format(z)
      !mkdir $d
      %cd $d 
      np.save("ECG_orig"+"{}_#{}".format(x,z),o)
      np.save("ECG_target"+"{}_#{}".format(y,z),t)




**Boundary Attack Black Box**

In [ ]:
# ATTACKING THE PROCESSED DATA
%cd
%cd /content/drive/My Drive/ECG_v1


for i in range(4):
  for j in range(4):
    if(i == j):
      continue
    for k in range(10):
      title = "ECG_orig{}_target{}".format(i,j) +"/" + "ECG_Pair{}".format(k)
      !python attack.py --input_dir ./PairedDataPhysio/$title/ --store_dir ./results_8000_Physio/$title/ --net ./ecg_mit.hdf5 --max_steps 8000 --s --candidate
    

**Processing Paired Data to be attacked with PhysioNet White Box**

In [ ]:
# install previous versions only for white box

!pip install https://github.com/mind/wheels/releases/download/tf1.8-cpu/tensorflow-1.8.0-cp36-cp36m-linux_x86_64.whl
!pip install keras==2.2
!pip install cleverhans==2.1


In [ ]:
# Creating a list with ALL INDICES OF ALL THE ORIGINAL ATTACK PAIRS FROM THE SOURCE CSV
# each element of list contains: name(ECG_orig{}_target{}/ECG_Pair{}, original ECG index in agg_data, and original ECG index in agg_data)

import scipy.io
from glob import glob
import numpy as np
saved = []
from natsort import natsorted

agg_data = np.load("/content/drive/My Drive/ECGadv-master/ProcessedData/agg_data.npy").reshape((8529,9000))
agg_data = agg_data[:,:1000]

for i in range(4):
  for j in range(4):
    if(i == j):
      continue
    for k in range(10):
      title = "ECG_orig{}_target{}".format(i,j) +"/" + "ECG_Pair{}".format(k)
      files = natsorted(glob("/content/drive/My Drive/ECG_v1/PairedDataPhysio" + "/" + title + "/*.npy"))
      individualSaved = [title]
      for a in files:
        # arrays has 2 files, original ECG and target ECG (paths)
        print(a)
        arr = np.load(a)
        arr = np.squeeze(arr)[:1000]
        index = (agg_data == arr).all(axis=1).argmax()
        print(index)
        individualSaved.append(index)
      saved.append(individualSaved)

# Ordered list, saved[0] contains the name and two files used in the attack,
# 1st element is 0 -> 1, pair1
# 2nd file is the original index, 3rd file is the target index     
print(len(saved))
np.save("/content/drive/My Drive/ECG_v1/IndicesData.npy",saved)



In [ ]:
import numpy as np

x= np.load("/content/drive/My Drive/ECG_v1/IndicesData.npy")
x= np.array(x)
# print(x)


In [ ]:
# create dictionary with title and corresponding original ECG data set
dict_index_title = {}
z = 1
for i in range(0,120):
  orig = int(x[i][1])
  title = x[i][0]
  dict_index_title.update({title:orig})

print(len(dict_index_title))

120


In [ ]:
dicti = {
     "0" : "data_select_A.csv",
     "1" : "data_select_N.csv",
     "2" : "data_select_O.csv",
     "3" : "data_select_i.csv",
}

In [ ]:
#WHITE BOX ATTACK CODE

import csv
import scipy.io

for i in range(120)):
  orig = x[i][1]
  orig = int(orig)
  print(orig)

  title = x[i][0]

  csvIndex = dicti[title[12:13]]
  print(csvIndex)
  
  csvfile = np.array(list(csv.reader(open("/content/drive/My Drive/ECGadv-master/" + csvIndex))))[:,3]                                                                                    
  csvfile = [int(i) for i in csvfile]
  csvfile = np.array(csvfile)
  
  index = np.where(csvfile==orig)[0][0]
  print(index)
  print(csvfile[index])
  
  index1 = index
  index2 = index + 1
 

  %cd
  %cd /content/drive/My Drive/ECGadv-master

  #smooth white box attack
  !python ./cloud_eval_diff.py $csvIndex $index1 $index2

  #l2 white box attack
  #!python ./cloud_eval_l2.py $csvIndex $index1 $index2


